In [1]:
import os
import sys
sys.path.append('..')

from scripts.modeling.evaluation import *
from scripts.modeling.model_selection import *
from scripts.data_utils.loaders import load_data, save_data
from scripts.data_utils.cleaning import convert_date_column
from scripts.modeling.preprocessing import split_data, preprocess_data, scale_features, time_series_split_data
from scripts.data_utils.feature_engineering import generate_risk_metrics, generate_features, generate_lag_and_rolling_features, apply_cyclical_encoding

%load_ext autoreload
%autoreload 2

In [2]:
# file_path = r"../data/AlphaCare_Risk_Insurance_data_Risk_ProfitMargin_processed.csv"
# processed_df = load_data(file_path)
# df = processed_df
# df.shape

c:\Users\Darkles\10academy\10Academy-Kifiya-Week-3\notebook\..\scripts\data_utils\loaders.py:8: DtypeWarning: Columns (4,32,37) have mixed types. Specify dtype option on import or set low_memory=False.
  dataframe = pd.read_csv(file_path, sep=sep)


Data loaded successfully.


(618464, 54)

In [3]:
# # Apply preprocessing
# df_preprocessed = preprocess_data(df)
# temp_df = df
# df = df_preprocessed
# df.shape

<class 'pandas.core.frame.DataFrame'>
Index: 618464 entries, 592855 to 398675
Columns: 1664 entries, UnderwrittenCoverID to IsWeekend_True
dtypes: float64(1658), int32(6)
memory usage: 7.7 GB


(618464, 1664)

In [4]:
# # dvc add data/AlphaCare_Risk_Insurance_data_All_processed.csv
# # git commit -am "Update dataset with new features"
# # dvc push 
# # dvc diff

# output_path = r'../data/AlphaCare_Risk_Insurance_data_All_processed.csv'
# save_data(df, output_path)

In [ ]:
file_path = r"../data/AlphaCare_Risk_Insurance_data_All_processed.csv"
processed_df = load_data(file_path)
df = processed_df
df = convert_date_column(df)
df.shape

In [5]:
# Split data into training and testing sets

# target_variable = 'ProfitMargin'
# target_variable = 'Risk'
# drop_columns = ['TotalClaims', 'TotalPremium'] # For  Risk and ProfitMargin

# target_variable = 'TotalPremium'
target_variable = 'TotalClaims'
drop_columns = ['Risk', 'ProfitMargin'] # For  TotalClaims and TotalPremium

df_feed = df.drop(columns=drop_columns, axis=1)
X_train, X_test, y_train, y_test = split_data(df_feed, target_variable=target_variable)

In [6]:
X_train_scaled, X_test_scaled = scale_features(X_train, X_test)

In [7]:
# Model training
lr_model = train_linear_regression(X_train_scaled, y_train)

In [8]:
rf_model = train_random_forest(X_train, y_train)

In [28]:
xgb_model = train_xgboost(X_train, y_train)

In [ ]:
evaluate_model(lr_model, X_test_scaled, y_test)

In [ ]:
evaluate_model(lr_model, X_test, y_test)

In [ ]:
evaluate_model(rf_model, X_test, y_test)

In [ ]:
# Model evaluation
models = {"Linear Regression": lr_model, "Random Forest": rf_model, "XGBoost": xgb_model}

for name, model in models.items():
    print(f"Evaluating {name}:")
    print(evaluate_model(model, X_test, y_test))
    print(evaluate_model(model, X_test_scaled if name == "Linear Regression" else X_test, y_test))

In [35]:
feature_names = X_train.columns

In [ ]:
model = xgb_model
# model = rf_model
feature_importance(model, feature_names)

In [ ]:
feature_importance_shap(model, X_train, feature_names=None)

In [ ]:
# Feature importance analysis using SHAP for Random Forest
feature_importance_shap_tree(rf_model, X_train, feature_names=feature_names)

In [ ]:
feature_importance_lime(model, X_train, feature_names, sample_idx=0)

In [ ]:
# Hyperparameter tuning
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'random_state': [42]
}
tuned_rf_model, best_params = hyperparameter_tuning(RandomForestRegressor(), param_grid, X_train, y_train)

evaluate_model(tuned_rf_model, X_test, y_test)

In [ ]:
evaluate_model(rf_model, X_test, y_test)

# Time-series data

In [ ]:
df = apply_cyclical_encoding(df)

In [ ]:
processed_data = generate_lag_and_rolling_features(df, date_column="TransactionMonth", target_column="TotalClaims")

In [ ]:
X_train, X_test, y_train, y_test = time_series_split_data(processed_data, "TransactionMonth", "TotalClaims", "2015-07-01")